## 混部集群性能分析
    混部：在线电商+离线ODPS作业
    
    在线电商：容器化/JAVA应用等，数量好几万
    离线ODPS作业：每天几十万-几百万作业
    
    
    本脚本为一个集群（AY49C）的混部性能数据
    
    
    fuxi_job：https://dmc.dw.alibaba-inc.com/dm/odps-table/odps.ali_meta.odps_fuxijob/
    fuxi_instance：https://dmc.dw.alibaba-inc.com/dm/odps-table/odps.meta.m_fuxi_instance/

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
%matplotlib inline
import matplotlib.pyplot as plt

sys.path.append("../../../")

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('max_colwidth',4000)
pd.options.display.expand_frame_repr = False

%load_ext autoreload
%autoreload 2
import time


In [ ]:
from pets.utils.logger import logger
from pets.utils.common import df_shape
from pets.core.data_loader.odps_acl.res_eff_acl_odps import odps, get_odps_table, \
get_odps_table_without_ds, write_odps_table
from pets.core.data_loader.odps_acl.general_acl_odps import get_odps_table as pets_get_odps_table,\
get_odps_table_without_ds as pets_get_odps_table_without_ds, write_odps_table as pets_write_odps_table


In [ ]:
from pets.core.data_loader.res_eff.data_source import get_vm_pdf, get_nc_pdf
from pets.core.data_loader.walle.data_source import get_walle_odps_host_cpu_util, get_walle_online_offline_cpu_util, get_walle_odps_container_cpu_util
import pets.core.data_loader.sunfire.data_source as sunfire
import pets.core.data_loader.kmon.data_source as kmon
import pets.core.data_loader.fuxi.data_source as fuxi

In [ ]:
cluster_group_name = ["phyhost-ecs_na610_ay49c"] # 主机分组
# cluster_group_name = ["ay49c_tubo_server"] # kata 容器分组

### Node Info

In [ ]:
nc_pdf = get_nc_pdf(ds=dates, app_group_names=cluster_group_name)

In [ ]:
nc_sn = nc_pdf.nc_sn.unique()

In [ ]:
nc_pdf.T

In [ ]:
nc_ips = nc_pdf.nc_ip.unique().tolist()
pod_pdf = get_vm_pdf(ds=dates, nc_ips=nc_ips)

In [ ]:
nc_ips

In [ ]:
# dates = ["20220209", "20220210"]
# start_time = '2022-02-09 00:00:00'
# end_time = '2022-02-11 00:00:00'
dates = ["20220206", "20220207", "20220208", "20220209", "20220210", "20220211", "20220212"]
start_time = "2022-02-06 00:00:00"
end_time = "2022-02-13 00:00:00"

fuxi.get_fuxi_instance_df() 当指定了start_time时ds只能有一个日期，否则拉不到数据

In [ ]:
eval_fuxi_instance_pdf = fuxi.get_fuxi_instance_df(ds=dates, start_time=start_time, end_time=end_time)
eval_fuxi_instance_pdf = eval_fuxi_instance_pdf[eval_fuxi_instance_pdf.worker_address.slice(6, -6).isin(target)]
#     eval_fuxi_instance_pdf = eval_fuxi_instance_pdf[['fuxi_job_name', 'fuxi_task_name',
#            'start_time', 'end_time','worker_address', 
#            'fuxi_avg_cpu', 'fuxi_avg_mem']]
#     eval_fuxi_instance_pdf.head(1)
# fuxi_job_name = eval_fuxi_instance_pdf.fuxi_job_name.to_pandas().tolist()
# print(fuxi_job_name)

# eval_fuxi_instance_pdf = fuxi.get_fuxi_instance_df(ds=["20220209", "20220210"], fuxi_job_name=eval_fuxi_instance_pdf.fuxi_job_name)
eval_fuxi_instance_pdf = eval_fuxi_instance_pdf[['fuxi_job_name', 'fuxi_task_name',
       'start_time', 'end_time','worker_address', 
       'fuxi_avg_cpu', 'fuxi_max_cpu', 'fuxi_avg_mem', 'fuxi_max_mem']]
eval_fuxi_instance_pdf.head(1)
eval_fuxi_instance_pdf = eval_fuxi_instance_pdf.to_pandas()
print(eval_fuxi_instance_pdf.shape)
worker = eval_fuxi_instance_pdf.worker_address.values
temp = []
for j in worker:
    temp.append(j[6:-6])
eval_fuxi_instance_pdf["worker_address"] = temp
eval_fuxi_instance_pdf.sort_values(by=["worker_address", "start_time"]).to_csv("/data/zkp/buy2/instance.csv", index=0)

In [ ]:
pod_meta.request_cpu.value_counts()

In [ ]:
for i in task_name:
    if i[0] == 'R' and len(i)>4:
        print(i)

In [ ]:
# pod_pdf[pod_pdf[pod_pdf.nc_sn.isin(nc_sn)].container_app_group_name=="ay49c_tubo_server"]
# target = pod_pdf[pod_pdf.nc_sn.isin(nc_sn)]
# target = target[target.container_app_group_name=="ay49c_tubo_server"]
target = pod_pdf[(pod_pdf.container_app_name=="ay49c") & (pod_pdf.nc_sn.isin(nc_sn))].container_ip.tolist()

In [ ]:
eval_fuxi_instance_pdf = fuxi.get_fuxi_instance_df(ds=["20220206"], project_name=["aliyun_searchlog"])
eval_fuxi_instance_pdf = eval_fuxi_instance_pdf[eval_fuxi_instance_pdf.status=="Terminated"]
eval_fuxi_instance_pdf["job_task_name"] = eval_fuxi_instance_pdf.fuxi_job_name + eval_fuxi_instance_pdf.fuxi_task_name
eval_fuxi_instance_pdf = eval_fuxi_instance_pdf[["job_task_name", "fuxi_job_name", "fuxi_task_name", "odps_task_type", "start_time", "end_time", "fuxi_avg_cpu",
                                                "fuxi_max_cpu", "fuxi_avg_mem", "fuxi_max_mem", "total_time_seconds"]].to_pandas()

In [ ]:
eval_fuxi_instance_pdf.to_csv("/data/zkp/recurrent/d0.csv", index=False)
# eval_fuxi_instance_pdf

In [ ]:
# pod_pdf = pd.read_csv("./pod_meta.csv")
pod_pdf[pod_pdf.nc_sn.isin(pod_pdf[pod_pdf.container_ip.isin(container_ip)].nc_sn)].to_csv("/data/zkp/buy2/5.19/pod_meta.csv", index=False)

In [ ]:
pod_pdf[pod_pdf.nc_sn==pod_pdf[pod_pdf.container_ip=="11.28.217.2"].nc_sn.values[0]]

In [ ]:
nc_sn = pod_pdf[pod_pdf.container_app_group_name=="buy2host"].nc_sn.unique().tolist()
len(nc_sn)

In [ ]:
ump2 = machine_meta[(machine_meta.container_app_group_name=="ump2_koalaguidehost") & machine_meta.nc_sn.isin(nc_sn)].nc_sn.unique().tolist()
ump2

In [ ]:
pod_pdf[pod_pdf.nc_sn.isin(nc_sn)].groupby("nc_sn").count().sort_values(by=["container_ip"])

In [ ]:
pod_pdf[pod_pdf.nc_sn=="i-8vb2cpnlzlyt7pi9fukj"].T

In [ ]:
container_ip = pod_pdf[pod_pdf.nc_sn=="i-8vb2cpnlzlyt7pi9fukj"].container_ip.unique().tolist()

In [ ]:
# container_ip = pod_pdf[(pod_pdf.container_app_group_name=="ump2_detailhost")].container_ip.unique().tolist()
# container_ip = machine_meta[machine_meta.nc_sn.isin(nc_sn)].container_ip.unique().tolist()
len(container_ip)

In [ ]:
pod_meta = pd.read_csv("/data/zkp/buy2/5.19/pod_meta.csv")

In [ ]:
container_ip = pod_meta.container_ip.unique().tolist()

In [ ]:
service = sunfire.get_sunfire_odps_df(ds=["20220519"], ips=container_ip, start_time="2022-05-19 00:20:00", end_time="2022-05-19 02:00:00")
service = service[service.throughput_type=="HSF"]
# service.sort_values(by=["container_ip", "sample_time__m", "throughput_type"]).to_csv("/data/zkp/buy2/service/allservice2.csv", index=False)

In [ ]:
service.sort_values(by=["container_ip", "sample_time__m", "throughput_type"]).to_csv("/data/zkp/buy2/5.19/allservice.csv", index=False)

In [ ]:
service

In [ ]:
d = ["20220217", "20220218", "20220219"]
target = pd.read_csv("./machine/metadata.csv")
target = target[target.nc_ip=="33.9.158.218"]
container_ip = list(target["container_ip"].values)
#     container_ip
d0 = sunfire.get_sunfire_odps_df(ds=[d[0]], ips=container_ip, start_time=start_time, end_time=end_time)
d1 = sunfire.get_sunfire_odps_df(ds=[d[1]], ips=container_ip, start_time=start_time, end_time=end_time)
d2 = sunfire.get_sunfire_odps_df(ds=[d[2]], ips=container_ip, start_time=start_time, end_time=end_time)

# # target_machine.head(20)

In [ ]:
pd.concat([d0, d1, d2]).sort_values(by=["container_ip", "sample_time__m", "throughput_type"]).to_csv("/data/zkp/m2_service.csv")

### 离线作业性能数据
    ay49c_tubo_server 为离线分组

### 离线作业性能数据
    ay49c_tubo_server 为离线分组

In [ ]:
odps_kata_ips = pod_pdf[pod_pdf["container_app_group_name"]=="ay49c_tubo_server"].container_ip.unique().tolist()
pod_pdf[pod_pdf.container_app_group_name=="ay49c_tubo_server"].to_csv("./machine/ay49c.csv")

In [ ]:
# fuxi jobs
eval_fuxi_job_pdf = fuxi.get_fuxi_job_df(ds=["20220210"], start_time="2022-02-10 00:00:00", end_time="2022-02-11 00:00:00")
eval_fuxi_job_pdf = eval_fuxi_job_pdf[["plan_cpu_cost", "plan_mem_cost"]]
eval_fuxi_job_pdf = eval_fuxi_job_pdf.to_pandas()
# eval_fuxi_job_pdf.to_csv("/data/zkp/buy2/5.12/job.csv")
# eval_fuxi_job_pdf.sum()

In [ ]:
eval_fuxi_job_pdf["cpu_mem"] = eval_fuxi_job_pdf.plan_cpu_cost.astype(str) + '+' + eval_fuxi_job_pdf.plan_mem_cost.astype(str)

In [ ]:
eval_fuxi_job_pdf.cpu_mem.value_counts().head(20) / eval_fuxi_job_pdf.cpu_mem.value_counts().sum()

In [ ]:
eval_fuxi_job_pdf.plan_cpu_cost.value_counts().head(20) / eval_fuxi_job_pdf.plan_cpu_cost.value_counts().sum()

In [ ]:
eval_fuxi_job_pdf.plan_mem_cost.value_counts().head(20) / eval_fuxi_job_pdf.plan_mem_cost.value_counts().sum()

In [ ]:
fuxi_inst_num[fuxi_inst_num.fuxi_inst_num>1000].fuxi_inst_num.sum() / fuxi_inst_num.fuxi_inst_num.sum()

In [ ]:
eval_fuxi_job_pdf = fuxi.get_fuxi_job_df(ds=["20220206", "20220207", "20220208", "20220209", "20220210", "20220211", "20220212"], 
                                         project_name=["aliyun_searchlog"]).to_pandas()

In [ ]:
eval_fuxi_job_pdf.to_csv("/data/zkp/recurrent/job.csv", index=False)
# eval_fuxi_job_pdf.drop(columns=["project_name", "odps_inst_id", "odps_task_name", "real_cpu_cost", "real_mem_cost", "end_time", "status", "fuxi_cpu_cost", "fuxi_mem_cost", "plan_gpu_cost", "nvm_used"], inplace=True)

In [ ]:
# np.savetxt("/data/zkp/figure/overview_2.txt" ,eval_fuxi_job_pdf.values.tolist())
eval_fuxi_job_pdf[(eval_fuxi_job_pdf.fuxi_task_num==11) & (eval_fuxi_job_pdf.fuxi_inst_num>4000)].sort_values(by=["fuxi_inst_num"]).fuxi_job_name.tolist()

In [ ]:
for k in range(10, 15):
    target_job = pd.read_csv("./taskDAG/task"+str(int(k))+".0.csv")
    mp = {}
    for i in range(target_job.shape[0]):
        if target_job["fuxi_job_name"][i] not in mp:
            mp[target_job["fuxi_job_name"][i]] = ""
        mp[target_job["fuxi_job_name"][i]] += target_job["fuxi_task_name"][i]
    eval_fuxi_job_pdf = fuxi.get_fuxi_job_df(ds=dates,
                                            cluster_name=["ay49c"],
                                            start_time=start_time,
                                            end_time=end_time, 
                                            fuxi_job_name=target_job.fuxi_job_name.tolist()).to_pandas()
    eval_fuxi_job_pdf.insert(7, "fuxi_task_name", "0")
    for i in range(eval_fuxi_job_pdf.shape[0]):
        if eval_fuxi_job_pdf["fuxi_job_name"][i] in mp:
            eval_fuxi_job_pdf.iloc[i, 7] = mp[eval_fuxi_job_pdf["fuxi_job_name"][i]]
    eval_fuxi_job_pdf.to_csv("./taskDAG/job"+str(int(k))+".csv")

In [ ]:
target_job = pd.read_csv("./taskDAG/task1.0.csv")
mp = {}
for i in range(target_job.shape[0]):
    if target_job["fuxi_job_name"][i] not in mp:
        mp[target_job["fuxi_job_name"][i]] = ""
    mp[target_job["fuxi_job_name"][i]] += target_job["fuxi_task_name"][i]
mp

In [ ]:
# eval_fuxi_job_pdf = pd.read_csv("./fuxi_job.csv")
# eval_fuxi_job_pdf.drop(columns=["project_name","running_cluster","real_cpu_cost","real_mem_cost","plan_gpu_cost","ds"], inplace=True)
drop_index = eval_fuxi_job_pdf[eval_fuxi_job_pdf["status"] != "Terminated"].index
terminated_fuxi_job = eval_fuxi_job_pdf.drop(drop_index)

In [ ]:
failed_fuxi_job = eval_fuxi_job_pdf[eval_fuxi_job_pdf["status"] != "Terminated"]

In [ ]:
eval_fuxi_job_pdf[eval_fuxi_job_pdf["odps_inst_id"] == "20220205230330908gjwnh1"].head(30)
#eval_fuxi_job_pdf.fuxi_job_sn.value_counts()

In [ ]:
eval_fuxi_job_pdf["start_time"] = eval_fuxi_job_pdf["start_time"] % 86400
#eval_fuxi_job_pdf.head(30)

In [ ]:
eval_fuxi_job_pdf["end_time"] = eval_fuxi_job_pdf["end_time"] % 172800

In [ ]:
eval_fuxi_job_pdf.sort_values("fuxi_inst_num", inplace=True, ascending=False)
eval_fuxi_job_pdf.head(50)

In [ ]:
fuxi_job_names = eval_fuxi_job_pdf.fuxi_job_name.tolist()

In [ ]:
fuxi_job_names = terminated_fuxi_job.head(150000).fuxi_job_name.unique().tolist()

In [ ]:
fuxi_job_names[:10]

In [ ]:
# fuxi task/fuxi_instance_df
for i in range(0, 150000, 10000):
    eval_fuxi_instance_pdf = fuxi.get_fuxi_instance_df(ds=dates,
                                                       fuxi_job_name=fuxi_job_names[i:i+10000],
                                                       start_time=start_time,
                                                       end_time=end_time)
    eval_fuxi_instance_pdf = eval_fuxi_instance_pdf[['fuxi_job_name', 'fuxi_task_name']].distinct()
    eval_fuxi_instance_pdf = eval_fuxi_instance_pdf.to_pandas()
    print(eval_fuxi_instance_pdf.shape)
    eval_fuxi_instance_pdf.to_csv("./taskDAG/task"+str(i/10000)+".csv")

33.39.230.200
33.4.113.88
33.4.140.198
33.4.149.57
33.4.153.135
33.4.177.205
33.4.202.236
33.4.210.125
33.4.85.113
33.4.89.204
33.4.93.71
33.4.96.107
33.4.96.71
33.49.58.73
33.61.106.159

In [ ]:
# fuxi sensor

In [ ]:
eval_fuxi_instance_pdf

In [ ]:
fuxi_sensor_df = fuxi.get_fuxi_sensor_df(ds=dates,
                                              start_time=start_time,
                                              end_time=end_time,
                                              fuxi_job_name=fuxi_job_names[:100],
                                              cluster_name=['AY49C'])

fuxi_sensor_df = fuxi_sensor_df.groupby(["hostname", "jobname", "rolename", "name"]) \
                                        .agg(
                                        fuxi_sensor_df["time"].max().rename("timestamp"),
                                        fuxi_sensor_df["system_cpu_cgroup_usage_nano"].sum().rename("cpu_time__ns"),
                                        fuxi_sensor_df["system_mem_cgroup_cache"].sum().rename("system_mem_cgroup_cache"),
                                        fuxi_sensor_df["system_mem_cgroup_rss"].sum().rename(
                                            "system_mem_cgroup_rss"),
                                        fuxi_sensor_df["system_mem_cgroup_total"].sum().rename(
                                            "system_mem_cgroup_total"),
                                    )
fuxi_sensor_df["cpu_time__ms"] = fuxi_sensor_df["cpu_time__ns"] / 1e3

fuxi_sensor_df = fuxi_sensor_df[
    fuxi_sensor_df.hostname.rename("nc_hostname"),
    fuxi_sensor_df.jobname.rename("fuxi_job_name"),
    fuxi_sensor_df.rolename.rename("fuxi_task_name"),
    fuxi_sensor_df.name.rename("fuxi_worker_name"),
    "timestamp",
    "cpu_time__ms",
    "system_mem_cgroup_cache",
    "system_mem_cgroup_rss",
    "system_mem_cgroup_total"
]

fuxi_sensor_pdf = fuxi_sensor_df.to_pandas()

In [ ]:
fuxi_sensor_pdf

### Node的性能数据

In [ ]:
host_cpu_util_pdf = get_walle_odps_host_cpu_util(ds=dates, start_time=start_time, end_time=end_time, nc_sn = nc_sn.tolist()).groupby(["sample_time"]).mean().to_pandas()

In [ ]:
host_cpu_util_pdf.sort_values(by=["nc_sn", "sample_time"]).to_csv("/data/zkp/buy2/5.19/machine.csv")

In [ ]:
host_cpu_util_pdf = host_cpu_util_pdf.sort_values(by=["sample_time"])

In [ ]:
plt.plot(range(host_cpu_util_pdf.shape[0]), host_cpu_util_pdf.cpu_util__pct_mean.values)

In [ ]:
temp = []
for i in range(1644076800, 1644681570, 3600):
    df = host_cpu_util_pdf[(host_cpu_util_pdf.sample_time>=i) & (host_cpu_util_pdf.sample_time<i+3600)]
    temp.append(df.cpu_util__pct_mean.mean())

In [ ]:
np.savetxt("/data/zkp/figure/overview_cpu.txt", temp)

In [ ]:
from pets.core.data_loader.walle.data_source import get_walle_odps_host_cpu_util,get_walle_online_offline_cpu_util, get_walle_odps_container_cpu_util

In [ ]:
walle_online_offline_cpu_util = get_walle_online_offline_cpu_util(ds=["20220519"], 
                                                                  start_time="2022-05-19 00:00:00", end_time="2022-05-19 02:00:00", nc_sns=nc_sn).to_pandas()

In [ ]:
walle_online_offline_cpu_util.sort_values(by=["Task_Type", "nc_sn", "sample_time"]).to_csv("/data/zkp/buy2/5.19/onoffline.csv")

In [ ]:
# pod_meta = pd.read_csv("/data/zkp/pod_meta.csv")
# nc_sn = pod_meta.nc_sn.unique()
nc_sn.shape

In [ ]:
temp = pod_pdf[pod_pdf.nc_sn.isin(["i-8vb9ul5xec4svv1bv1h0", "i-8vb4p5hyvakr5me0hij1", 
"i-8vb9qrifimm00x67wj12",
"i-8vbcaknvxr6wbba8tlsu",
"i-8vb2t1xpt6kzgz2brw9h",
"i-8vb13matvy02bw36lch3",
"i-8vb9bx27cup4tevcgur7",
"i-8vbf565jz2vjsriw73ke",
"i-8vb19vzmoywz7xqtuvdz"])]

In [ ]:
nc_sn = pod_pdf[pod_pdf.container_app_group_name=="buy2host"].nc_sn.unique().tolist()
ips = pod_pdf[pod_pdf.nc_sn.isin(nc_sn)].container_ip.unique().tolist()
len(nc_sn), len(ips)

In [ ]:
pod_meta = pd.read_csv("/data/zkp/buy2/5.19/pod_meta.csv")

In [ ]:
container_ip = pod_meta[pod_meta.container_app_group_name=="buy2host"].container_ip.values

In [ ]:
nc_sn = pod_meta[pod_meta.container_app_group_name=="buy2host"].nc_sn.values

In [ ]:
container_ip.shape, nc_sn.shape

In [ ]:
service = sunfire.get_sunfire_odps_df(ds=["20220609"], ips=container_ip.tolist(), start_time="2022-06-09 20:00:00", end_time="2022-06-09 21:30:00")
service = service[service.throughput_type=="HSF"]
# service = service[["sample_time__m", "container_ip", "rt__ms_Q"]]

In [ ]:
service2 = sunfire.get_sunfire_odps_df(ds=["20220524"], ips=container_ip.tolist(), start_time="2022-05-24 20:00:00", end_time="2022-05-24 22:00:00")
service2 = service2[service2.throughput_type=="HSF"]

In [ ]:
service3 = sunfire.get_sunfire_odps_df(ds=["20220519"], ips=container_ip.tolist(), start_time="2022-05-19 00:00:00", end_time="2022-05-19 01:30:00")
service3 = service3[service3.throughput_type=="HSF"]

In [ ]:
offline1 = get_walle_online_offline_cpu_util(ds=["20220609"], nc_sns=nc_sn.tolist(), start_time="2022-06-09 20:00:00", end_time="2022-06-09 21:30:00").to_pandas()
offline1 = offline1[offline1.Task_Type=="Offline"]

offline2 = get_walle_online_offline_cpu_util(ds=["20220524"], nc_sns=nc_sn.tolist(), start_time="2022-05-24 20:00:00", end_time="2022-05-24 22:00:00").to_pandas()
offline2 = offline2[offline2.Task_Type=="Offline"]

offline3 = get_walle_online_offline_cpu_util(ds=["20220519"], nc_sns=nc_sn.tolist(), start_time="2022-05-19 00:00:00", end_time="2022-05-19 01:30:00").to_pandas()
offline3 = offline3[offline3.Task_Type=="Offline"]

In [ ]:
DATA = pd.concat([service, service2, service3])

In [ ]:
offline = pd.concat([offline1, offline2, offline3])

In [ ]:
DATA.shape, service.shape, service2.shape, service3.shape

In [ ]:
offline.shape, offline1.shape, offline2.shape, offline3.shape

In [ ]:
DATA = DATA.astype({"sample_time__m": str})

In [ ]:
DATA["sample_time"] = DATA.sample_time__m.apply(lambda x:int(time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S"))))

In [ ]:
DATA = DATA.sort_values(by=["nc_sn", "sample_time"])
offline = offline.sort_values(by=["nc_sn", "sample_time"])

In [ ]:
DATA = pd.merge(DATA, pod_meta[["container_ip", "nc_sn"]], on=["container_ip"], how="left")

In [ ]:
offline

In [ ]:
DATA

In [ ]:
data = pd.merge(DATA, offline, on=["nc_sn", "sample_time"], how="left")

In [ ]:
data

In [ ]:
data = data[~pd.isna(data.cpu_util__pct_y) & (data.cpu_util__pct_y>0)]

In [ ]:
data = data[data.qps__Q_s>5]

In [ ]:
data = data[data.container_ip.isin(["33.38.56.97", "33.37.158.122", "33.38.49.195", "33.38.19.86", "33.38.15.217"])]

In [ ]:
data.to_csv("/data/zkp/figure/ob3_new.csv", index=None)

In [ ]:
from sklearn.linear_model import LinearRegression
for ip in ["33.38.56.97", "33.37.158.122", "33.38.49.195", "33.38.19.86", "33.38.15.217"]:
    fontsize=20
    ob3 = data[data.container_ip==ip]
    plt.figure(figsize=(8, 7), dpi=120)
    plt.scatter(ob3.cpu_util__pct_y/96, ob3.rt__ms_Q.values, marker="+", s=150, c="blue")
    plt.xlabel("BE CPU utilization (%)", fontsize=fontsize)
    plt.xticks(fontsize=fontsize)
    plt.ylabel("RT (ms)", fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.grid(ls=":")
    plt.show()
    model = LinearRegression()
    model.fit(ob3.cpu_util__pct_y.values.reshape(-1, 1)/96, ob3.rt__ms_Q.values)
    print(model.coef_, model.intercept_)

In [ ]:
import time
DATA["sample_time"] = DATA.sample_time__m.astype("str").apply(lambda x:int(time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S"))))

In [ ]:
DATA = pd.merge(DATA, walle_online_offline_cpu_util[["sample_time", "nc_sn", "CPU_time__s"]], how="left", on=["sample_time", "nc_sn"])

In [ ]:
DATA = DATA[["sample_time", "container_ip", "rt__ms_Q", "container_app_group_name", "nc_sn"]]

In [ ]:
DATA.nc_sn.unique().shape

In [ ]:
# corr1 = []
corr2 = []
buy2 = DATA[(DATA.container_app_group_name=="ump2_detailhost") & (DATA.CPU_time__s>1000)]
for ip in buy2.container_ip.unique():
    temp = buy2[buy2.container_ip==ip]
    if temp.shape[0] > 2000:
        corr2.append(temp.rt__ms_Q.corr(temp.CPU_time__s))

In [ ]:
plt.plot(sorted(corr1), np.arange(len(corr1))*100 / (len(corr1) - 1), lw=4, c="blue")
plt.plot(sorted(corr2), np.arange(len(corr2))*100 / (len(corr2) - 1), lw=4, c="orange")

In [ ]:
pod_meta.container_app_group_name.unique().shape

In [ ]:
app = ['accs-sal_newretailhost', 'busphost', 'buy2host', "ump2_detailhost",
       'content-managerhost', 'lubanservicehost', 'mediaplatformhost',
       'rhino-dcc-wssyshost', 'tblive-prodhost',
       'ayfc-fc-zjk-internal-workers', 'cloudvideohost', 'lippi-atwhost',
       'lippi-dingflowpmchost', 'lippi-dingpayhost',
       'merchant-rating-portal_hz_host', 'sdphost',
       'tianshu-vpc_center_host', 'verified_host',
       'wdk-infrastructure_dumphost', 'buy2_damai_host', 'alsc-buy2host',
       'alsc-promo-centerhost', 'alsc-svip-producthost',
       'alsc-ump_guidehost', 'fc-usergrowth-fronthost', 'iliadhost',
       'lippi-mixhost', 'livekithost', 'mclarenhost',
       'mmc-scm-purchasehost', 'op-one-auth_serverlesshost', 'rdamhost',
       'wdk-plugin-engine-301879host', 'op-datawar_serverless_host',
       'ai-ssp-otahost', 'alsc-consumehost', 'alsc-contenthost',
       'alsc-dish-queryhost', 'alsc-rights_prod', 'alsc-rights_prod_send',
       'alsc-timeout-centerhost', 'ataphost', 'auctionsearchhost',
       'birdsnestcenter_prehost', 'havanamloginhost',
       'lippi-industry-businesshost', 'livex-righthost',
       'na61-digitalstoreapi', 'skyway-sophost', 'tdsettlehost',
       'timeoutcenterhost', 'trippchost', 'alihealth-crm-chainhost',
       'traindpp_grap_prehost', 'alsc-ad-marketinghost',
       'alsc-svip-orderhost', 'alscadpisceshost', 'aselfcaptainhost',
       'goc-mineralhost', 'jst-trade-guardhost', 'keludeweb_greyhost',
       'mmc-cartshost', 'mozi-conf-apphost', 'oculus-beiruthost',
       'union-topapihost', 'industry-platform-adminhost',
       'intl-planning_host_zb', 'fbt-online_prehost',
       'alihouse-new-homehost', 'cogmanhost',
       'global-uic-s_center_ae-hz_host',
       'nbf-online-opt_default_projecthost', 'rateplatformhost',
       'saber_kgb', 'sharepasswordhost', 'travelitemsmanagerhost',
       'alsc-upp-corehost', 'bbqhost', 'inventory_xiaoerhost',
       'kbt-wordrphost', 'lippi-industry-homeworkhost',
       'settle-centerhost', 'ts-spatial-engine_4.0.0',
       'union-biz-model-ophost', 'nbf-online-c2m_default_projecthost',
       'atorhost', 'buyer-work-product_hz_host', 'copperhost',
       'cpc-tracehost', 'hicchost', 'hpsshost', 'htdshost',
       'hz.atmtaskserver', 'iedatahost', 'lippi-open-financehost',
       'navhost', 'nbpickuphost', 'tmallxsearch', 'tomort-saashost',
       'wireless-content-service-od_host', 'ad-feedhost',
       'aicloud-gateway-bizservicehost', 'alsc-buyhost', 'asure_hz_host',
       'atickethost', 'cngfchost', 'ele-biz-anteaterhost',
       'ele-newretail-open-gatewayhost', 'livex-showhost',
       'mmc-fulfill-promisehost', 'mmc-ofchost', 'orighthost',
       'profilesynchost', 'ump2_wudaokou_host', 'vcphost', 'waimai-carts',
       'settle-schedulehost', 'tmsx-decision_dpathprehost',
       'health-bot-17host', 'alidataservice_hz_host',
       'cf-payment-core_rthost', 'cnbp_sidehost',
       'ele-newretail-trade-service-prehost', 'hjbillingserver_prehost',
       'lark-itemprodhost', 'lippi-doc-cloud-notehost',
       'lippi-security-xtee-client_asynchost', 'mec-imkthost',
       'minishophost', 'tddispatchhost', 'tdpowerhost',
       'quark-report-alibaba_hz_prehost',
       'alihealth-b2c-patient-icc_prehost', 'amdchost', 'cmploomhost',
       'cnskuhost', 'cnwirelesshost', 'ele-waimai-tmhost', 'peanuthost',
       'ele-newretail-bops_prehost', 'ai-agency-adminhost',
       'amp-relationhost', 'gearfactoryhost', 'linkstreamhost',
       'livex-growinghost', 'matrixexchangehost', 'mbis_guide_host',
       'tmsx-practicehost', 'tripcrowdhost', 'xspace-insighthost',
       'nbf-online-data_application_defaulthost',
       'alsc-consume-centerhost', 'banama_tp_host', 'coin-dm_gray4_host',
       'equidaehost', 'jym-layouthost', 'mec-searchhost', 'tmc-dumphost',
       'mec-tradesynchost', 'chongzhimobile_prehost', 'ali-commenthost',
       'alsc-item-queryhost', 'amp-sync2host',
       'aplatform-taobaolife-publish', 'general-ticketshost',
       'jstqa-centerhost', 'nbf-online-dchain-industry-fresh-invoicehost',
       'os-smartdq_os_cbu_auto_cbu_auto_ol_ol', 'ump2_retail_otherhost',
       'vt-orderprocesshost', 'wukong-relationhost',
       'ai-operation-centerhost', 'alibee-shophost', 'alsc-dmphost',
       'dosahost', 'dwaplushost', 'etao-rebate-servicehost',
       'kbt-channel-prodhost', 'ma-settlementhost',
       'wdk-walle-publisherhost', 'alsc-tmhost', 'gcbs_compliancehost',
       'lippi-checkinhost', 'lippi-event-centerhost',
       'nbf-online-309418host', 'powermsg-proxyhost',
       'sellerservice-basehost', 'tccp-mininghost', 'yuntaihost',
       'atpcx_prehost', 'alsc-upphost', 'amp2-im-receiverhost',
       'buc-acl-hsf_prehost', 'ic2_L1_host', 'imagebankhost',
       'lippi-livehost', 'tlenshost', 'towerhost',
       'union-payment-reporthost', 'ascp-goodshost',
       'capacitymanagerhost', 'dephost', 'ele-newretail-order-core_host',
       'hzunit-luna', 'kbtcdphost', 'lippi-thaleshost',
       'wukong-contactshost', 'wukong-registerhost', 'alsc-gccs-aoihost',
       'ai-dsphost', 'capacitymanager_centerhost', 'cngcphost',
       'inventoryschedulerhost', 'lippi-orghost',
       'lippi-template-centerhost', 'mmc-right-centerhost',
       'seller-libraryhost', 'upp-hubhost', 'lippi-classroomhost',
       'guangguang-producerhost', 'ic2_L0c_host', 'idlefunhost',
       'inventory_trade_writehost', 'live-controlhost',
       'live-interactive-centerhost', 'member-benefit-writehost',
       'mmc-markethost', 'mmc-target-externalhost', 'shidc-publish',
       'stationplatformhost', 'tmallsearchservicehost',
       'cncourier_betahost', 'einvoice-prodhost', 'fr-orderhost',
       'lightspeedhost', 'lst-umphost',
       'os-smartdq_os_wenyu_auto_wenyu_auto_ol_ol', 'tk-detailhost',
       'wiehost', 'wukong-notifyhost', 'welkinbhost', 'bombonerahost',
       'guangguang-seller-managerhost', 'mec-guide-whost',
       'topnotifyhost', 'tbdx-sc-trade_prehost', 'alilang-agentserver',
       'htmhost', 'hz.bwservice', 'trademanager_sellerhost', 'upphost',
       'vipshophost', 'ai-dsp-base_gray4_host', 'alsc-budget-centerhost',
       'banliang-acctranshost', 'nbf-online-dingtalk-recommend-userhost',
       'op-flowhost', 'ratemanagerhost', 'shellcenterhost', 'temaihost',
       'trip-mainhost', 'mev-datastat_prehost', 'asr-dataservice_host',
       'coin-dmhost', 'gaia-hd-ltao-publish', 'helpcenterhost',
       'iefthost', 'macenterhost', 'wukong-settinghost', 'datamindhost',
       'nbf-budgethost', 'nbf-online-perm-industryhost',
       'ogs-ticklet_prehost', 'accs-masshost',
       'aicloud-device-centerhost', 'alsc-order-queryhost',
       'beehive-detailhost', 'ic2_L2_host', 'lippi-dingflowbpmshost',
       'mp-strategyhost', 'paymentplatform3_cn_host',
       'quark-report__gray6_host', 'regerahost', 'xchathost', 'atpcmhost',
       'alsc-upp-eventhost', 'xt-biz-wishhost',
       'ai-ssp-device-centerhost', 'ald-lamphost', 'cnbp_onlinehost',
       'content-log-servicehost', 'hxs_corehost',
       'inventory_query_koala_host', 'refundplatform3host', 'tp3g0host',
       'fliggy-buy2_prehost', 'cngcp_aewh_dev', 'alsc-trade-centerhost',
       'ic2_cbu_host', 'lippi-safehost', 'tmap-data-statisticshost',
       'wprchost', 'alsc-mp-consult_guide', 'content-detailhost',
       'fether-bridgehost', 'idlelabelhost', 'settleplatform_g2host',
       'tc-gp-centerhost', 'tmc-apply_taskhost', 'upp-ugchost',
       'zsynchost', 'traingrap_yy_host', 'rex-itnet-bridgehost',
       'aecp-view-designer-vpc_center_host', 'cloudsailhost',
       'kbtcdp_prehost', 'tmall-camphost', 'gaia-pmapp-index-prepub',
       'ai-iot-platformhost', 'ak-testservicehost', 'auctionpolarishost',
       'goc-authhost', 'inventory_sell_writehost',
       'linkedmall-platformhost', 'lippi-microapphost', 'ltpphost',
       'tdalbatrosshost', 'os-scm-microservices_prehost', 'adp2host',
       'alisite-cbu-mobilehost', 'alsc-magiclamphost',
       'caigou-centerhost', 'cartscachehost', 'coingamehost',
       'etao-myhost', 'lippi-interactive-cardhost', 'wsmhost', 'aposhost',
       'rtee_prehost', 'agooservicehost', 'cuntaosettlehost',
       'dd-databushost', 'idlerealtimepushhost',
       'kbt-servindustry-store-detailhost', 'mallitemcenterhost',
       'pur-rcvhost', 'tfservicehost', 'uicfinalhost', 'umcchost',
       'epaas-acschost', 'cpsloginhost', 'creditonehost',
       'insa-benefit_host', 'lippi-fingerhost',
       'os-smartdq_os_ae_biz_auto_ae_biz_auto_ol_ol',
       'xspace-ivr_smartcallhost', 'alihealth-nr-tradehost',
       'alsc-marketing-calculate-centerhost', 'idle-localhost',
       'lsttrade-taskhost', 'ngalipmc_gray6_host',
       'tg-rpc_consign-platform_host', 'track_hzhost', 'xfile-commonhost',
       'iotx-linkcard-user-centerhost', 'alsc-item-detailhost',
       'alsc-rights_prod_consult', 'divide-media-centerhost',
       'hic_corehost', 'lippi-orgviphost', 'makerhost',
       'miniapp-metadatahost', 'rodimushost', 'rubickhost',
       'alsc-tag-servicehost', 'eticket2host', 'fliggytmhost',
       'idle-homehost', 'matrixexchange_notifyhost',
       'sellerservice-workbenchhost', 'xengine-automation_dispute_host',
       'xiaofancenterhost', 'xt-biz-quanhost',
       'cfo-df-data-asset_pre_grey', 'smart-oneprofile_gray6_host',
       'alidns-core-service-aliyun-comhost', 'lippi-industry-centerhost',
       'marketing-centerhost', 'wdk-scm-returnhost',
       'ahas-sentinel_inner_host', 'btxshost', 'idle-searchhost',
       'kbt-recsyshost', 'risk-disposal-centerhost', 'asc-nrshost',
       'knowledgecloud-olympic_prehost', 'alsc-cs-reversehost',
       'oneringhost', 'os-smartdq_os_ylxb_zb_auto_ylxb_zb_auto_ol_ol',
       'poihost', 'pricinghost', 'wdetailhost', 'alscdmptaskhost',
       'aicloud-content-bizhost', 'ic2_other_host', 'idledetailhost',
       'mp-ttshost', 'sdp_recommend_host',
       'ele-newretail-order-management_betahost', 'eticketasynchost',
       'imagebank-synchost', 'nbf-online-rex-login-post_defaulthost',
       'pumpkinhost', 'aliveplatformhost', 'alsc-live-showhost',
       'alsc-pos-organizationhost', 'ayfc-fc-hsf-gateway-prod-pouch',
       'catapulthost', 'godivahost', 'hse1host', 'idle-promotionshost',
       'trippoihost', 'wmpspacehost', 'aliad-stl-archimedeshost',
       'coveragedoctorhost', 'aboxhost', 'alime-base-olympichost',
       'auctionboxhost', 'ele-newretail-groot-merchanthost',
       'micdarwinhost', 'mudphost',
       'nbf-online-permission_tdi_resourcehost',
       'wenxue-aggregate-datahost', 'xspace-mcu_smartcallhost',
       'center-customercenter.interface', 'giuliahost', 'livex-vcorehost',
       'tripadslhost', 'alsc-svip-memberhost', 'bakkhoshost',
       'cco-data-servicehost_lindorm', 'favorite-server_hz-intl_host',
       'jubuithost', 'lark-marketinghost', 'lippi-payhost',
       'retail-cloud-corehost', 'tfgatewayhost', 'tmg-user-matrixhost',
       'message-platformhost', 'accountlinkhost', 'alime-nluhost',
       'alsc-aladdinhost', 'lippi-at-powerxhost',
       'mmc-logisticsdetailhost', 'odin-data-mininghost',
       'tmall-farmhost', 'wallethost', 'c2m-cost_prehost',
       'alisite-taobao-serverhost', 'alsc-merchant-queryhost', 'atichost',
       'ctcorehost', 'lippi-wfcollectorhost', 'tdtradeplatformhost',
       'tfinteractionhost', 'tforderhost', 'ultramanhost',
       'shield-kgbhost', 'alsc-rec-data-centerhost',
       'idleapi-messagehost', 'motormasterhost', 'lippi-planethost',
       'adplus-site-subway', 'cargohost', 'crowdhost', 'grapplehost',
       'lippi-conversation-toolbarhost', 'mcmsadminservice_hz_host',
       'wdk-fulfill-time-expressionhost', 'xt-biz-zhibohost',
       'ad-feed-fliggy', 'agooackhost', 'alsc-fulfillment-platform2host',
       'idleitem_host', 'nbf-online-scitem_abilityhost',
       'pos-customerhost', 'rhone-adminhost', 'alicom-dsfhost',
       'alphaerhost', 'triptphost', 'refundplatform3_pre13host',
       'lippi-transcribehost', 'alime-shop-gatewayhost', 'alsc-passhost',
       'apos_g1host', 'charming2host', 'et2-janusweb-pub',
       'gaia-tjb-growth-synstate-publish', 'lippi-genie-accesshost',
       'o2o-ad-exposure-serverhost', 'auctionbarrierhost',
       'beehive-dataapihost', 'csp-commonhost', 'livex-funhost',
       'tradeapihost', 'ump2_retail_normalhost',
       'iot-ide-execute_2_prehost', 'aeye-juno-connector-innerhost',
       'alilang-uem-dpchost', 'alsc-langrisser-dump_prehost',
       'bpbsev_kgb', 'gaea-applicationhost', 'lippi-circlehost',
       'performhost', 'taobao-rpghost', 'wegame-xxl-bizhost',
       'risk-strategy-managementhost', 'qnproviderhost',
       'campus-workbench_prehost', 'ele-newretail-drug_prehost',
       'sdp_platform_prehost', 'itemcenterschedule2host',
       'malldetailskiphost', 'topdatapush-cloudhost', 'trippshost',
       'scmbporder-lst_prehost', 'alifocushost', 'bentleyhost_bc',
       'offerqueryhost', 'wdk-plugin-engine-201949host',
       'cloudwifi-collectorhost',
       'ascp-procurement_international_prehost',
       'fliggy-service-platformhost', 'grandethost', 'lippi-todohost',
       'alsc-refund-platformhost', 'porschehost', 'x4vhost',
       'mz-market_yyh_prehost', 'ihome-itemcenter-store_prehost',
       'ald-lamphost_biz-20200110', 'gpf-item_api_host', 'idlecenterhost',
       'lippi-at-calculatorhost', 'lippi-bihost', 'lippi-doc-memohost',
       'userportalhost', 'alsc-user-centerhost', 'buc-acl-hsfhost',
       'data-riderhost', 'idle-luxuryhost', 'lippi-doc-bizhost',
       'mkt-turbohost', 'tg-rpc_tianchi_host', 'uicplusdatasynchost',
       'vilyahost', 'waterfallhost', 'xspace-record_smartcall',
       'nbf-pre-alihealth-brand-toufang_prehost', 'gpf4-retailhost',
       'paganihost', 'tmc-rulehost', 'gaia-aplatform-ddz-publish',
       'agoo-server-apihost', 'alsc-consumer-queryhost',
       'beehive-scenes_mphost', 'cco-data-servicehost', 'gameapihost',
       'hz.arcadia', 'idle-mtop-searchhost', 'wmpoperate-check_betahost',
       'xspace-intl-as-service_beta', 'ald-lamphost-dacu',
       'membercenterhost', 'nbf-online-hr-gwhost', 'pointcenterhost',
       'taesitehost', 'xcloud-pfmhost', 'service-product-centerhost',
       'wdk-scm-schedulehost', 'alsc-sales-batchhost', 'alphaer_prehost',
       'alsc-item-centerhost', 'einvoice-uahost',
       'hz.magellan_openservice', 'inventoryplatformhost',
       'nbcolumbahost', 'xspace-record_smartcall_prehost',
       'ai-iot-device-centerhost', 'ald-sensorhost',
       'amp-relation-imhost', 'cornucopiahost', 'ic2_fliggy_host',
       'lippi-org-centerhost', 'lippi-tdphost',
       'nbf-online-ticket-attributehost', 'union-mrchost',
       'hjsettle-ability_cn_taskhost', 'lippi-doc-corehost',
       'nextrpc-decouplerhost', 'qualitydatacenterhost',
       'trip-order-center-dumphost', 'ump2_guidehost', 'hr-cbbp_uat',
       'flycphost', 'intl-infinity-servicehost', 'logisticsdetailhost',
       'topnotify_apihost', 'ump2_detailhost', 'waimai-item-basichost',
       'tradeplatform3_pre_order', 'tfkapalai_prehost',
       'alicom-dsf_edenhost', 'coding-biz-notifyhost', 'mmc-plutushost',
       'mp-trackerhost', 'bi-businesshost_sycm', 'pentaprismhost',
       'placebusinesshost', 'xengine-knowledge-olympichost',
       'ai-model-enginehost', 'copper_prehost', 'mmc-user-centerhost',
       'recommend-pro-maxhost', 'site_service_centerhost',
       'xsop_tbfirst_host', 'op-datawarhost', 'punirserver_hz_prehost',
       'alsc-cs-autumnhost', 'alsc-rec-data-center_prod1',
       'baoxianriskhost', 'fr-platformhost', 'jupushcenterhost',
       'lippi-workspacehost', 'wdk-riskhost', 'mozi-conf-app_opshost',
       'brigehost', 'cn-account_customer_host', 'einvoice-corehost',
       'ele-lpd-infra-tsd-grid-zbprod-zb1', 'lippi-eam_betahost',
       'mmc-logistics-basichost', 'mos-asset-platformhost',
       'remuscenter-taobao-host', 'sicshost', 'ricky_prehost',
       'mafiahost', 'alsc-cartshost', 'cngdf_b2b_host',
       'havanaservicehsfhost', 'hmqhost', 'message-platformhost_ucp',
       'one-scn-supply-network_prehost', 'pcs-order-searchhost',
       'ump2_ump3_retail_otherhost', 'huangjincehost', 'nbaquilahost',
       'tripwmchost', 'iotx-linkcard-consolehost',
       'tjb-seller-service-centerhost', 'alihealth-vaccinhost', 'cushost',
       'lippi-search_viphost', 'tfitemcenterhost', 'topapi_itemhost',
       'aecp-view-designer-vpc_center_prehost',
       'uplus-corehost_dataset_grp_run', 'etao-message-taskhost',
       'kbtservindustryhost', 'lst-poshost', 'one-scn_prehost',
       'wdk-plugin-engine-302136host', 'wwhongbaohost', 'nbehost',
       'umplus-trustid-serverhost', 'mobius-qa_pre2host',
       'ald-gatewayhost', 'c2m-inventory_hz_host',
       'nbf-pre-rex-login-role_prehost',
       'alisite-taobao-server_storehost', 'aplatformhost',
       'ele-newretail-haseehost', 'falcon9r-data-factoryhost',
       'gpf4-sell_upload_host', 'usc-taskhost',
       'wdk-plugin-engine-303465host', 'ae-ut-crab-s_center_na610_host',
       'chongzhimobilehost', 'etao-message-centerhost', 'wheelie_host',
       'xt-biz-etaohost', 'tmsx-dispatch_betahost', 'itfp-loan_prehost',
       'cne-orderhost', 'cnt-finance-billhost', 'coincorehost',
       'creditpayhost', 'crmcenterhost',
       'ele-newretail-shop-query_stablehost', 'forge_innerhost',
       'global-tracking-service_host', 'mkt-mission_spehost',
       'mmc-recommendhost', 'xspace-ehomehost', 'traingraphost',
       'acc-oeden_prehost', 'hpchost', 'shopcenterhost',
       'privacy-write-service_hz_prehost', 'union-common-docshost',
       'alsc-pos-orderhost', 'ascm-saas-merchandisehost',
       'ele-biz-growth-task-core-zbprod-zb1', 'idleapihost',
       'ump2_normalhost', 'uplus-adminhost', 'lippi-omphost',
       'wdk-plugin-engine-301911host', 'ascp-cost-gundamhost',
       'alsc-marketing-platformhost', 'alsc-user-growth-enginehost',
       'center-rights.center', 'wdk-dms-metricshost',
       'xengine-knowledgehost', 'lippi-collab-needlehost', 'ispyhost',
       'alsc-marketing-dynamic-promotion', 'fwsite_youku_host',
       'hic_tophost', 'kbt-industry-operation_prehost', 'live-socialhost',
       'trippehost', 'dutyfree-goods_prehost', 'ald-gateway_tmcshphost',
       'c2m-costhost', 'lippi-contentcenterhost', 'trademanagerhost',
       'volvohost', 'wmpitemctr_gw_host',
       'linkedmall-itemadminservicehost', 'ald-lamphost-diantao',
       'alsc-ump_normalhost', 'amdc_openhost', 'm-self-servicehost',
       'mec-markethost', 'tripcrm-catchhost', 'union-cps-relationhost',
       'miniapp-cloudhost', 'topapihost', 'tmg-customer-ophost',
       'alscadscorpiohost', 'ccormshost', 'mc-throneshost', 'osphost',
       'wdk-plugin-engine-302036_prehost', 'alsc-data-scannerhost',
       'ascp-risk-managementhost', 'lark-umprodhost', 'rteehost',
       'settleplatform_g1host', 'tblive-itemhost',
       'union-collaborationhost', 'scn-distribution-line-command_prehost',
       'ald-lamp_prehost-rmc1', 'cngcp_ae_host', 'hz.ensearchweb',
       'mmc-item-facadehost', 'musehost', 'panama-oceanhost',
       'promotiontaghost', 'idms_prehost',
       'ops-ipoll-server_serverless_host', 'wdk-scm-return_prehost',
       'ieu-scene-buildhost', 'alsc-activityhost',
       'alsc-marketing-calculate-center_guide', 'cngdf_ae_host',
       'cngtouchhost', 'ecardmisccenterhost',
       'gaia-zebra-handler-index-publish', 'mbishost',
       'nbf-online-ascp-purchasing-anti_host', 'alsc-umpprodhost',
       'ic2_L0c_gray4_host', 'ecology-org_prehost',
       'alsc-delivery-servicehost', 'etao-halo-fehost', 'lippi-dinghost',
       'op-sycm-aehost', 'paymentplatform3_pre25_host', 'powermsghost',
       'shopactcenterhost', 'solar-crowd-engine_publish',
       'mmc-network-pps_prehost', 'mec-ugshost',
       'productcenterservice_hz_cloud', 'smartui-gatewayhost',
       'tariffcodehost', 'thor-materialhost', 'wdkmarkethost',
       'mmc-interact-centerhost', 'tp3g2host', 'quickbi-sycmhost',
       'finance-reporthost', 'alsc-cs-summerhost', 'alsc-rule-centerhost',
       'ascp-charge-rulehost', 'cf-acquirerhost', 'ffatouchhost',
       'kbrouteprodhost', 'mmc-crm-crowdhost', 'panama-strategyhost',
       'ump2_koalahost', 'coin-lunyu_pre2', 'bbq-dumphost',
       'clouddisk_hz_host', 'dwd-basedata-service_prehost',
       'mkt-shellhost', 'protectorhost', 'trade-receipthost',
       'xt-biz-wkhost', 'tfitemfacadehost',
       'aicloud-share-center_1_prehost', 'goc-cosophost']